# 运行单个管道步骤

本笔记本将介绍如何通过单个管道步骤（AssignWCS 和 FlatField）来校准数据，而不是运行整个管道阶段。

**用例**：当使用标准 JWST 管道以外的包时，某些步骤仍然可以在 JWST 管道中使用。在这里，我们展示最常运行的单个步骤，AssignWCS 和 FlatField。AssignWCS 确定并存储 WCS（世界坐标系统）信息，而 FlatField 则去除 [探测器特征](https://jwst-docs.stsci.edu/known-issues-with-jwst-data/niriss-known-issues#NIRISSKnownIssues-artifactsArtifacts)。<br>

**数据**：来自程序 2079 的 JWST/NIRISS 图像和光谱。<br>

**工具**：astropy, crds, glob, jwst, matplotlib, numpy, os, urllib, zipfile<br>

**跨仪器**：NIRISS<br>

**内容**

- [导入与数据设置](#imports)

- [运行单个管道步骤](#pipeline_steps)

  - [Assign WCS 步骤](#wcs_step)

  - [平场步骤](#ff_step)

  - [比较率与平场数据](#compare)

**作者**：Rachel Plesha (rplesha@stsci.edu), Camilla Pacifici (cpacifici@stsci.edu)<br>

**最后修改**：2024年5月<br>

**最后测试**：本笔记本最后在 JWST 管道版本 1.12.5 和 CRDS 上下文 jwst_1229.pmap 下进行测试。

<a id='imports'></a>

## 导入与数据设置

In [ ]:
# 更新CRDS路径到你的本地目录
%env CRDS_PATH=crds_cache  # 设置CRDS缓存路径为'crds_cache'

# 设置CRDS服务器的URL
%env CRDS_SERVER_URL=https://jwst-crds.stsci.edu  # 设置CRDS服务器的URL为JWST CRDS的地址

In [ ]:
import os  # 导入操作系统模块，用于文件和目录操作

import glob  # 导入glob模块，用于文件路径匹配

import urllib  # 导入urllib模块，用于处理URL

import zipfile  # 导入zipfile模块，用于处理ZIP文件

import numpy as np  # 导入NumPy库，用于数值计算和数组操作

from astropy.io import fits  # 从Astropy库导入fits模块，用于处理FITS文件格式

from matplotlib import pyplot as plt  # 从matplotlib库导入pyplot模块，用于绘图

%matplotlib inline  # 在Jupyter Notebook中内嵌显示绘图

from jwst.assign_wcs import AssignWcsStep  # 从jwst库导入AssignWcsStep，用于分配世界坐标系统

from jwst.flatfield import FlatFieldStep  # 从jwst库导入FlatFieldStep，用于平场校正

检查您正在使用的JWST（詹姆斯·韦伯太空望远镜）管道版本。要查看可用的最新管道版本或安装以前的版本，请访问 [GitHub](https://github.com/spacetelescope/jwst#software-vs-dms-build-version-map)。同时，请确认您正在使用的 [CRDS（参考数据系统）上下文](https://jwst-crds.stsci.edu/)。 [CRDS文档](https://jwst-pipeline.readthedocs.io/en/latest/jwst/user_documentation/reference_files_crds.html) 解释了如何在JWST管道中设置特定的上下文。如果这两个值与上述最后测试的说明不同，则此笔记本中可能会存在差异。

In [ ]:
import jwst  # 导入JWST相关的库

import crds  # 导入CRDS（Calibration Reference Data System）库

print('JWST Pipeliene Version:', jwst.__version__)  # 打印JWST管道的版本信息

print('CRDS Context:', crds.get_context_name('jwst'))  # 获取并打印JWST的CRDS上下文名称

#### 数据设置

在这里，我们下载并打开包含所有速率文件的压缩文件，并且如果输出目录尚不存在，我们还会创建一个用于存放校准文件的输出目录。

In [ ]:
import os  # 导入os模块，用于处理文件和目录

data_dir_in = 'data'  # 输入目录，存放速率文件的目录

data_dir_out = 'data/calibrated_steps/'  # 输出目录，存放校准文件的目录

# 如果要保存的目录不存在，则首先创建该目录
for datadir in [data_dir_in, data_dir_out]:  # 遍历输入和输出目录列表

    if not os.path.exists(datadir):  # 检查目录是否存在

        os.makedirs(datadir)  # 如果不存在，则创建该目录

In [ ]:
# 从Box下载未校准的数据到数据目录：

boxlink = 'https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/niriss_wfss_advanced/niriss_wfss_extra_input.zip'  # Box文件的链接

boxfile = os.path.basename(boxlink)  # 获取文件名

urllib.request.urlretrieve(boxlink, boxfile)  # 下载文件

zf = zipfile.ZipFile(boxfile, 'r')  # 打开下载的zip文件

zf.extractall(path=data_dir_in)  # 解压文件到指定的数据输入目录

# 将从Box文件下载的文件移动到顶级数据目录

box_download_dir = os.path.join(data_dir_in, boxfile.split('.zip')[0])  # 构建下载目录路径

for filename in glob.glob(os.path.join(box_download_dir, '*')):  # 遍历下载目录中的所有文件

    if '.csv' in filename:  # 如果文件是CSV格式

        # 移动到当前目录

        os.rename(filename, os.path.basename(filename))  # 重命名并移动到当前目录

    else:  # 如果文件不是CSV格式

        # 移动到数据目录 

        os.rename(filename, os.path.join(data_dir_in, os.path.basename(filename)))  # 重命名并移动到数据目录

# 现在删除不必要的文件

os.remove(boxfile)  # 删除下载的zip文件

os.rmdir(box_download_dir)  # 删除下载目录

<a id='pipeline_steps'></a>

## 运行单个管道步骤

虽然您可以查看速率图像，但建议您将文件通过管道的 `assign_wcs` 和 `flat_field` 步骤进行处理，以清理探测器伪影。

<a id='wcs_step'></a>

#### 指定WCS步骤

管道的 `assign_wcs` 步骤是获取WFSS图像正确光谱轨迹切片的关键部分。要了解更多关于该步骤的信息，请访问 [AssignWCS描述页面](https://jwst-pipeline.readthedocs.io/en/latest/jwst/assign_wcs/main.html)。

In [ ]:
# 运行 assign_wcs；这里我们仅对一个文件进行演示

# 定义输入文件的路径，使用 os.path.join 连接目录和文件名
ratefile = os.path.join(data_dir_in, 'jw02079004002_02101_00001_nis_rate.fits')

# 调用 AssignWcsStep 的 call 方法，处理 ratefile 文件，并指定输出目录和保存结果的选项
result = AssignWcsStep.call(ratefile, output_dir=data_dir_out, save_results=True)

In [ ]:
# 快速的合理性检查，以确保文件已被校准。

# 如果这个值为零，请检查上面的日志消息以获取在校准过程中可能发生的任何错误

wcsstep_files = glob.glob(os.path.join(data_dir_out, '*assignwcsstep*'))  # 获取指定目录下所有包含'assignwcsstep'的文件

print(len(wcsstep_files), 'assignwcsstep files written')  # 打印已写入的assignwcsstep文件数量，应该有1个文件

<a id='ff_step'></a>

#### 平场步骤 (Flat Field Step)

在运行 assignwcs 文件后，我们接下来要运行管道的 `flat_field` 步骤，该步骤使用平场参考文件去除探测器伪影。要了解更多关于该步骤的信息，请访问 [平场描述页面](https://jwst-pipeline.readthedocs.io/en/latest/jwst/flatfield/main.html)。

In [ ]:
# 运行平场校正

for wcsfile in wcsstep_files:  # 遍历所有的WCS文件

    result = FlatFieldStep.call(wcsfile, output_dir=data_dir_out, save_results=True)  # 调用平场校正步骤，保存结果到指定输出目录

In [ ]:
# 快速的合理性检查，以确保文件已被校准。

# 如果这个值为零，请检查上面的日志消息以查看在校准过程中是否发生了任何错误

flatfield_files = glob.glob(os.path.join(data_dir_out, '*flatfieldstep*'))  # 获取所有匹配'flatfieldstep'的文件

print(len(flatfield_files), 'flatfieldstep files written')  # 打印写入的'flatfieldstep'文件数量，应该有1个文件（与wcs步骤匹配）

<a id='compare'></a>

#### 比较速率数据与平场校正数据

运行下面的单元格可以显示来自速率文件的相同直接图像，与运行管道的 `flat_field` 步骤后的图像相比。一些探测器伪影明显消失，例如探测器右下角和中间的 [交叉哈希](https://jwst-docs.stsci.edu/known-issues-with-jwst-data/niriss-known-issues#NIRISSKnownIssues-Cross-hatching)。

在 [NIRISS 已知问题页面](https://jwst-docs.stsci.edu/known-issues-with-jwst-data/niriss-known-issues) 上还有一些需要注意的光学伪影，例如 1/f 噪声。

In [ ]:
test_rate_file = ratefile  # 将ratefile赋值给test_rate_file，以便查看直接图像进行比较

test_flat_file = os.path.join(data_dir_out, os.path.basename(test_rate_file).replace('rate.fits', 'flatfieldstep.fits'))  # 构建平场校正文件的路径

plot_files = [test_rate_file, test_flat_file]  # 创建一个包含要绘制的文件的列表

plot_titles = ['Rate File', 'Flat Corrected File']  # 创建一个包含对应标题的列表

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 15))  # 创建一个包含两个子图的图形

fig.suptitle(f"{os.path.basename(test_rate_file).split('_rate')[0]}\n{fits.getval(test_rate_file, 'PUPIL')}", x=0.5, y=0.72)  # 设置图形的总标题

for filename, title, ax in zip(plot_files, plot_titles, [ax1, ax2]):  # 遍历文件、标题和子图

    with fits.open(filename) as hdu:  # 打开FITS文件

        # 用零填充坏像素的nan值；否则，单个未抖动的图像将无法真正显示

        data = hdu[1].data  # 获取数据部分

        data[np.isnan(data)] = 0  # 将nan值替换为0

        

        ax.imshow(data, vmin=0.2, vmax=1.2, origin='lower')  # 显示图像，设置颜色范围和原点位置

        ax.set_title(title)  # 设置子图标题

<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="太空望远镜标志" width="200px"/>